# Disaster Tweets with Long-Short Term Memory Networks

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" alt="Drawing" style="width: 250px;"/>

Welcome to my Disaster Tweet LSTM notebook, where we'll try to train a LSTM (or *Long-Short Term Memory*) neural network on the disaster tweets to predict whether they are real or not. This notebook produced a public score of 0.81, which is pretty good for starters.

You can notice that I am using _various data sources_ for this notebook. That's because I used the Gensim Word Embedding dataset featured here: https://www.kaggle.com/iezepov/gensim-embeddings-dataset. These are basically pre-trained embeddings (that is, word->vector of real numbers mappings) based on a huge number of documents by the state-of-the-art people at Google, Stanford, Facebook and the such. We will exploit these instead of training embeddings ourselves to save time (and also, they work better).

Let's start by importing all the required libraries and setting variables:

In [ ]:
# Math and linear algebra
import numpy as np

# Data manipulation
import pandas as pd

# Plots
import seaborn as sns
import matplotlib.style as style
import matplotlib.pyplot as plt

# Keras: high-level neural network API 
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate, Reshape
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.preprocessing import text, sequence

# Sk-learn
from sklearn.feature_extraction.text import CountVectorizer

# Gensim: NLP library. We will use this to load the word embedding vectors
from gensim.models import KeyedVectors

import re

# Keras: deep learning API
import keras
print(keras.__version__)
import tensorflow
print(tensorflow.__version__)

# Embedding files
EMBEDDING_FILES = [
    '../input/gensim-embeddings-dataset/crawl-300d-2M.gensim',
    '../input/gensim-embeddings-dataset/glove.840B.300d.gensim',
    '../input/gensim-embeddings-dataset/GoogleNews-vectors-negative300.gensim'
]

# 
BATCH_SIZE = 512
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
EPOCHS = 20
MAX_LEN = 220
MAX_WORDS = 27
TEXT_COLUMN = 'text'
TARGET_COLUMN = 'target'
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'

We imported gensim to deal with the Embeddings dataset, as we'll see later. The rest are the usual Kaggle competition imports :-).

Then we set up a number of "constants" (actually variables) which we are going to use in the rest of the notebook. First we set up the embedding datasets we are going to use: in this case, I used fasttext's crawl dataset trained on Common Crawl and Wikipedia (https://fasttext.cc/docs/en/crawl-vectors.html) and GloVe, trained by Stanford on the same sources (https://nlp.stanford.edu/projects/glove/).

Then I set the batch size, the number of LSTM units, number of dense hidden units, epochs, max sequence length (220 characters since it's Twitter), and some chars to remove from the tweets.

Now let's load the data and see what we have.

In [ ]:
print("Read Data")
train_df = pd.read_csv('../input/nlp-getting-started/train.csv')
test_df = pd.read_csv('../input/nlp-getting-started/test.csv')
train_df.head(5)

So we have an id column (with missing numbers, which are the test id's), keyword, location, the tweet itself, and target - that is, whether it's an actual disaster or not.

Now let's do some analytics.

# EDA

In [ ]:
print("There are", train_df.shape[0], "tweets in the training dataset.")

We can see there are 7613 tweets, and that 30%+ of the locations are missing, as well as ~5% of keywords:

In [ ]:
style.use("ggplot")
x = train_df.isna().sum()*100.0/train_df.shape[0]
ax = sns.barplot(x.index, x, palette="Blues_d")
ax.set_ylabel("%")

In [ ]:
v = train_df["target"].value_counts()
sns.barplot(v.index, v)

In [ ]:
fig,axes=plt.subplots(1,2,figsize=(10,5), sharey='row')
fig.suptitle("Tweet length in characters")
for i in range(2):
    sns.distplot(train_df[train_df["target"] == i]["text"].str.len(), ax=axes[i], color=sns.color_palette()[i])
    axes[i].set_xlabel("Real = "+str(i))

In [ ]:
fig,axes=plt.subplots(1,2,figsize=(10,5), sharey='row')
fig.suptitle("Tweet length in words")
for i in range(2):
    sns.distplot(train_df[train_df["target"] == i]["text"].str.split().map(lambda x: len(x)), ax=axes[i], color=sns.color_palette()[i])
    axes[i].set_xlabel("Real = "+str(i))

In [ ]:
fig,axes=plt.subplots(1,2,figsize=(9,5))
fig.suptitle("NaNs by variable")
v = train_df["keyword"].isnull().groupby(train_df["target"]).sum().astype(int)
sns.barplot(v.index, v, ax=axes[0])
axes[0].set_ylabel("# of NaNs")
axes[0].set_xlabel("")
axes[0].set_title("Keyword")

v = train_df["location"].isnull().groupby(train_df["target"]).sum().astype(int)
sns.barplot(v.index, v, ax=axes[1])
axes[1].set_ylabel("# of NaNs")
axes[1].set_xlabel("")
axes[1].set_title("Location")

fig.tight_layout(rect=[0, 0.03, 1, 0.90])

In [ ]:
# Most common words
from collections import Counter
fig, axes = plt.subplots(1, 2, figsize=(14,5))
fig.suptitle("Most common words")
for i in range(2):
    word_list = [x for sublist in list(train_df[train_df["target"] == i]["text"].map(lambda x: x.split()).values) for x in sublist]
    top = sorted(Counter(word_list).items(), key=lambda x:x[1], reverse=True)[:10]
    x, y = zip(*top)
    axes[i].bar(x, y)
    axes[i].set_xlabel("Real = "+str(i))

In [ ]:
x_train = train_df[TEXT_COLUMN].astype(str)
y_train = train_df[TARGET_COLUMN].values
x_test = test_df[TEXT_COLUMN].astype(str)

tokenizer = text.Tokenizer(filters=CHARS_TO_REMOVE, lower=False)
tokenizer.fit_on_texts(list(x_train) + list(x_test))

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)

# Model compilation, training and prediction

In [ ]:
checkpoint_predictions = []
weights = []

def build_matrix(word_index, path):
    embedding_index = KeyedVectors.load(path, mmap='r')
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        for candidate in [word, word.lower()]:
            if candidate in embedding_index:
                embedding_matrix[i] = embedding_index[candidate]
                break
    return embedding_matrix

embedding_matrix = np.concatenate(
    [build_matrix(tokenizer.word_index, f) for f in EMBEDDING_FILES], axis=-1)

def build_model(embedding_matrix):
    words = Input(shape=(MAX_LEN,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True, dropout=0.2))(x)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True, dropout=0.2))(x)
    #x = Dropout(0.4)(x)
    #res = Reshape((-1, x_train.shape[1], 100))(x)
    #conv2 = Conv2D(100, (3,3), padding='same',activation="relu")(x)
    #pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    #flat = Flatten()(pool2)
    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x)
    ])
    dense = Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)
    dense = Dense(DENSE_HIDDEN_UNITS, activation='relu')(dense)
    result = Dense(1, activation='sigmoid')(dense)
    model = Model(inputs=words, outputs=result)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
print("Start Modeling")
model = build_model(embedding_matrix)
model.summary()
model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    verbose=1
)
print("Modeling Complete")

In [ ]:
predictions = model.predict(x_test)
submission = pd.DataFrame.from_dict({
    'id': test_df.id,
    TARGET_COLUMN: np.round(predictions).astype(int).reshape(x_test.shape[0])
})
submission.to_csv('lstm_submission.csv', index=False)
submission.head()